In [8]:
import pandas as pd

data = pd.read_csv('./old_data/old_positive_data_2092_elem.csv', header=0, sep=',')
print(data.shape)
print(data.dtypes)
data.head()

(2092, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...
1,LinJ.01,24093,25080,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...
2,LinJ.01,35371,36297,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...
3,LinJ.01,39790,40595,plus,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...
4,LinJ.01,54983,55909,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...


In [2]:
data_neg = pd.read_csv('./data/old_negative_data_691_elem.csv', header=0, sep=',')
print(data_neg.shape)
print(data_neg.dtypes)
data_neg.head()

(691, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.04,175855,176491,plus,CGCGTATGACGGCCGTGCGCCGACTCGGTCGGCTGAGCGCGCGGCC...
1,LinJ.04,208516,208737,plus,AAGCAGCTGCAGACACACGTGTGTGACGGCCGTGTGCCGACTCGGT...
2,LinJ.04,208748,209410,plus,CCTGACCCTGCTATGGCCAGGGGCGTTGCGGCGTTGGCGGGGATGA...
3,LinJ.04,331481,332011,plus,GTCGCTCTGACCTCCCCCTCTCACGCCCTAGGCACCCTGGCCCTGC...
4,LinJ.04,362873,363525,plus,CATCCGGATGACAGGTGGGGGAGGGGGTGCGCACACCTCAGCGCGT...


In [6]:
def get_length(x):
    return len(x)

In [9]:
data['length'] = data.apply(lambda x: get_length(x['sseq']), axis=1)
data.head()

,sseqid,sstart,send,sstrand,sseq,length
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...,1000
1,LinJ.01,24093,25080,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...,988
2,LinJ.01,35371,36297,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927
3,LinJ.01,39790,40595,plus,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...,806
4,LinJ.01,54983,55909,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,927


In [12]:
cmd = (
    f'blastdbcmd -db  '
    f'-entry '
    f'-range  '
    f'-strand  '
    f'-outfmt %s'
)
cmd

'blastdbcmd -db  -entry -range  -strand  -outfmt %s'